# 11.5 심층 강화 학습의 구현



리스트11.1 모듈의 임포트와 최적화 알고리즘의 설정

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import animation, rc

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, ReLU
from tensorflow.python.keras.optimizer_v2.rmsprop import RMSprop

optimizer = RMSprop()

리스트11.2 Brain 클래스

In [2]:
class Brain:
    def __init__(self, n_state, n_mid, n_action, gamma=0.9, r=0.99):
        self.eps = 1.0  # ε
        self.gamma = gamma  # 할인율
        self.r = r  # ε의 감쇠율

        model = Sequential()
        model.add(Dense(n_mid, input_shape=(n_state,)))
        model.add(ReLU()) 
        model.add(Dense(n_mid))
        model.add(ReLU()) 
        model.add(Dense(n_action))
        model.compile(loss="mse", optimizer=optimizer)
        self.model = model

    def train(self, states, next_states, action, reward, terminal):
        q = self.model.predict(states)  
        next_q = self.model.predict(next_states)
        t = np.copy(q)
        if terminal:
            t[:, action] = reward  #  에피소드 종류 시의 정답은 보수만
        else:
            t[:, action] = reward + self.gamma*np.max(next_q, axis=1)
        self.model.train_on_batch(states, t)

    def get_action(self, states):
        q = self.model.predict(states)
        if np.random.rand() < self.eps:
            action = np.random.randint(q.shape[1], size=q.shape[0])  # 랜덤한 행동
        else:
            action = np.argmax(q, axis=1)  # Q값의 높은 행동
        if self.eps > 0.1:  # ε의 하한
            self.eps *= self.r
        return action

리스트11.3 에이전트를 클래스로서 구현

In [3]:
class Agent:
    def __init__(self, v_x, v_y_sigma, v_jump, brain):
        self.v_x = v_x  # x 속도
        self.v_y_sigma = v_y_sigma  # y 속도, 초기값의 표준 편차
        self.v_jump = v_jump  # 점프 속도
        self.brain = brain
        self.reset()

    def reset(self):
        self.x = -1  # 초기 x 좌표
        self.y = 0  # 초기 y 좌표
        self.v_y = self.v_y_sigma * np.random.randn()  # 초기 y 속도

    def step(self, g):  # 시간을 1개 앞으로 움직인다  g:중력 가속도
        states = np.array([[self.y, self.v_y]])
        self.x += self.v_x
        self.y += self.v_y

        reward = 0  # 보수
        terminal = False  # 종료 판정
        if self.x>1.0:
            reward = 1
            terminal = True
        elif self.y<-1.0 or self.y>1.0:
            reward = -1
            terminal = True
        reward = np.array([reward])

        action = self.brain.get_action(states)
        if action[0] == 0:
            self.v_y -= g   # 자유 낙하
        else:
            self.v_y = self.v_jump  # 점프
        next_states = np.array([[self.y, self.v_y]])
        self.brain.train(states, next_states, action, reward, terminal)

        if terminal:
            self.reset()

리스트11.4 환경을 클래스로 구현

In [4]:
class Environment:
    def __init__(self, agent, g):
        self.agent = agent
        self.g = g

    def step(self):
        self.agent.step(self.g)
        return (self.agent.x, self.agent.y)

리스트11.5 물체의 비행을 애니메이션으로 나타내는 FuncAnimation() 함수

In [5]:
def animate(environment, interval, frames):
    fig, ax = plt.subplots()
    plt.close()
    ax.set_xlim(( -1, 1))
    ax.set_ylim((-1, 1))
    sc = ax.scatter([], [])

    def plot(data):
        x, y = environment.step()
        sc.set_offsets(np.array([[x, y]]))
        return (sc,)

    return animation.FuncAnimation(fig, plot, interval=interval, frames=frames, blit=True)

리스트11.6 에이전트가 랜덤으로 행동

In [6]:
n_state = 2  # 상태의 수
n_mid = 32  # 중간층의 뉴런 수
n_action = 2  # 행동의 수
brain = Brain(n_state, n_mid, n_action, r=1.0)  # ε이 감쇠하지 않는다

v_x = 0.05  # 수평 방향의 이동 속도
v_y_sigma = 0.1  # 초기 이동 속도(수직 방향)의 퍼지는 정도
v_jump = 0.2  # 점프 시의 수직 방향 속도
agent = Agent(v_x, v_y_sigma, v_jump, brain)

g = 0.2  # 중력 가속도
environment = Environment(agent, g)

interval = 50  # 애니메이션 순간(밀리초)
frames = 1024  # 프레임 수
anim = animate(environment, interval, frames)
rc('animation', html='jshtml')
anim

리스트11.7 DQN에 의한 학습

In [ ]:
n_state = 2  # 상태의 수
n_mid = 32  # 중간층의 뉴런 수
n_action = 2  # 행동의 수
brain = Brain(n_state, n_mid, n_action, r=0.99)  # ε이 감쇠한다

v_x = 0.05  # 수평 방향의 이동 속도
v_y_sigma = 0.1  # 초기 이동 속도(수직 방향)의 퍼지는 상태
v_jump = 0.2  # 점프 시의 수직 방향 속도
agent = Agent(v_x, v_y_sigma, v_jump, brain)

g = 0.2  # 중력 가속도
environment = Environment(agent, g)

interval = 50  # 애니메이션의 간격(밀리초)
frames = 1024  # 프레임 수
anim = animate(environment, interval, frames)
rc('animation', html='jshtml')
anim